### ДЗ к уроку №2 "TensorFlow" и распознавание жилья в Бостоне.
    
    Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

    Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

    Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

    1-2. (*) Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews .Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?


In [1]:
# import needed libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical

from sklearn.datasets import load_boston as load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

print('tensorflow version =',tf.__version__)

tensorflow version = 2.5.0


In [2]:
# loading the dataset and put in in train and test
X, y = load_boston(return_X_y=True)

In [3]:
data = pd.DataFrame(X)
data

0     1      2    3      4      5     6       7    8      9     10  \
0    0.00632  18.0   2.31  0.0  0.538  6.575  65.2  4.0900  1.0  296.0  15.3   
1    0.02731   0.0   7.07  0.0  0.469  6.421  78.9  4.9671  2.0  242.0  17.8   
2    0.02729   0.0   7.07  0.0  0.469  7.185  61.1  4.9671  2.0  242.0  17.8   
3    0.03237   0.0   2.18  0.0  0.458  6.998  45.8  6.0622  3.0  222.0  18.7   
4    0.06905   0.0   2.18  0.0  0.458  7.147  54.2  6.0622  3.0  222.0  18.7   
..       ...   ...    ...  ...    ...    ...   ...     ...  ...    ...   ...   
501  0.06263   0.0  11.93  0.0  0.573  6.593  69.1  2.4786  1.0  273.0  21.0   
502  0.04527   0.0  11.93  0.0  0.573  6.120  76.7  2.2875  1.0  273.0  21.0   
503  0.06076   0.0  11.93  0.0  0.573  6.976  91.0  2.1675  1.0  273.0  21.0   
504  0.10959   0.0  11.93  0.0  0.573  6.794  89.3  2.3889  1.0  273.0  21.0   
505  0.04741   0.0  11.93  0.0  0.573  6.030  80.8  2.5050  1.0  273.0  21.0   

         11    12  
0    396.90  4.98  
1    396.90  9.14  
2    392.83  4.03  
3    394.63  2.94  
4    396.90  5.33  
..      ...   ...  
501  391.99  9.67  
502  396.90  9.08  
503  396.90  5.64  
504  393.45  6.48  
505  396.90  7.88  

[506 rows x 13 columns]

In [4]:
data.describe()

0           1           2           3           4           5   \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.322453    6.860353    0.253994    0.115878    0.702617   
min      0.006320    0.000000    0.460000    0.000000    0.385000    3.561000   
25%      0.082045    0.000000    5.190000    0.000000    0.449000    5.885500   
50%      0.256510    0.000000    9.690000    0.000000    0.538000    6.208500   
75%      3.677083   12.500000   18.100000    0.000000    0.624000    6.623500   
max     88.976200  100.000000   27.740000    1.000000    0.871000    8.780000   

               6           7           8           9           10          11  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean    68.574901    3.795043    9.549407  408.237154   18.455534  356.674032   
std     28.148861    2.105710    8.707259  168.537116    2.164946   91.294864   
min      2.900000    1.129600    1.000000  187.000000   12.600000    0.320000   
25%     45.025000    2.100175    4.000000  279.000000   17.400000  375.377500   
50%     77.500000    3.207450    5.000000  330.000000   19.050000  391.440000   
75%     94.075000    5.188425   24.000000  666.000000   20.200000  396.225000   
max    100.000000   12.126500   24.000000  711.000000   22.000000  396.900000   

               12  
count  506.000000  
mean    12.653063  
std      7.141062  
min      1.730000  
25%      6.950000  
50%     11.360000  
75%     16.955000  
max     37.970000

In [5]:
labels = pd.Series(y)
labels

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Length: 506, dtype: float64

Для процесса обучения нейронной сети нам важно перевести данные из диапазона от 0 до 255 в диапазон от 0 до 1:

data = data / 255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
y_t = pd.DataFrame(y_test, columns=['target'])
y_t.reset_index(inplace=True)
y_test = y_t.drop(['index'], axis = 1)
y_test

## Построение модели

сделаем 3 разные модели и посмотрим какая лучше, разделение:
- по числу нейронов в скрытых слоях
- по активационной функции в скрытых слоях
- по оптимизатору

In [33]:
for j in ['adam', 'sgd', 'rmsprop']:
    for i in ['sigmoid', 'relu', 'tanh', 'linear']:
        # заполняем модель списком
        model1 = Sequential([
            Flatten(input_shape=(13,)),
            Dense(100, activation= i),
            Dense(90, activation= i),
            Dense(1, activation='linear')
            ])

        # заполняем модель , добавляя слои последовательно 
        model2 = Sequential()
        model2.add(Flatten(input_shape=(13,)))
        model2.add(Dense(200, activation= i))
        model2.add(Dense(190, activation= i))
        model2.add(Dense(1, activation='linear'))

        # Заполняем модель, формируя граф передачи тензоров
        x_input = Input(shape=(13,))
        x3 = Dense(300, activation= i)(x_input)
        x3 = Dense(290, activation= i)(x3)
        x_output = Dense(1, activation='linear')(x3)
        model3 = Model(x_input,x_output)

        model1.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])
        model2.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])
        model3.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])

        hist1 = model1.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)
        hist2 = model2.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)
        hist3 = model3.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)

        # Predictions of 3 models, target and MSE of each model:
        result1 = pd.DataFrame(model1.predict(X_test).flatten(), columns = ['model1'])
        result2 = pd.DataFrame(model2.predict(X_test).flatten(), columns = ['model2'])
        result3 = pd.DataFrame(model3.predict(X_test).flatten(), columns = ['model3'])

        # объединяем результаты моделей и добавляем цель
        Summary = pd.concat([result1, result2, result3, y_test], axis=1, join='inner')

        # вычисляем СКО для каждой модели и каждой цели
        Summary['MSE1'] = (Summary['model1'] - Summary['target'])**2
        Summary['MSE2'] = (Summary['model2'] - Summary['target'])**2
        Summary['MSE3'] = (Summary['model3'] - Summary['target'])**2

        # усредняем СКО для каждой модели
        MSE_model1 = Summary['MSE1'].mean()
        MSE_model2 = Summary['MSE2'].mean()
        MSE_model3 = Summary['MSE3'].mean()
        print('For 100 epochs:')
        print('MSE_model1 100 neuros = ', MSE_model1, ' for ', i, 'for ', j)
        print('MSE_model2 200 neuros = ', MSE_model2, ' for ', i, 'for ', j)
        print('MSE_model3 300 neuros = ', MSE_model3, ' for ', i, 'for ', j)

For 100 epochs:
MSE_model1 100 neuros =  39.99173686960522  for  sigmoid for  adam
MSE_model2 200 neuros =  26.403103681955567  for  sigmoid for  adam
MSE_model3 300 neuros =  21.40229234675259  for  sigmoid for  adam
For 100 epochs:
MSE_model1 100 neuros =  38.68485110514497  for  relu for  adam
MSE_model2 200 neuros =  26.84043195235355  for  relu for  adam
MSE_model3 300 neuros =  64.93213645835583  for  relu for  adam
For 100 epochs:
MSE_model1 100 neuros =  33.324386162539305  for  tanh for  adam
MSE_model2 200 neuros =  21.06569254359124  for  tanh for  adam
MSE_model3 300 neuros =  26.2975195223722  for  tanh for  adam
For 100 epochs:
MSE_model1 100 neuros =  80.92333703269776  for  linear for  adam
MSE_model2 200 neuros =  139.43125514278685  for  linear for  adam
MSE_model3 300 neuros =  79.21468495657751  for  linear for  adam
For 100 epochs:
MSE_model1 100 neuros =  35.50363474810986  for  sigmoid for  sgd
MSE_model2 200 neuros =  25.834343469214325  for  sigmoid for  sgd
MS

### Вывод: лучшее MSE  =  21.065 получили для случая: 200 neuros  for activation = tanh  for opt = adam
Далее изучаем многослойность для этого варианта

Посмотрим как получается при изменении количества слоёв: 2, 3, 4

In [34]:
# создаем модели
model1 = Sequential([ 
                      Flatten(input_shape=(13,)),
                      Dense(200, activation='tanh'),
                      Dense(190, activation='tanh'),
                      Dense(1, activation='linear'),
                     ])
model2 = Sequential([
                      Flatten(input_shape=(13,)),
                      Dense(200, activation='tanh'),
                      Dense(190, activation='tanh'),
                      Dense(180, activation='tanh'),
                      Dense(1, activation='linear'),
                      ])
model3 = Sequential([
                      Flatten(input_shape=(13,)),
                      Dense(200, activation='tanh'),
                      Dense(290, activation='tanh'),
                      Dense(280, activation='tanh'),
                      Dense(270, activation='tanh'),
                      Dense(1, activation='linear'),
                      ])
model4 = Sequential([ 
                      Flatten(input_shape=(13,)),
                      Dense(200, activation='tanh'),
                      Dense(1, activation='linear'),
                     ])


model1.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])
model2.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])
model3.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])
model4.compile(optimizer='adam',
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])

hist1 = model1.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)
hist2 = model2.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)
hist3 = model3.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)
hist4 = model4.fit(data, labels, epochs=100, validation_split=0.2, verbose=0)

# Predictions of 3 models, target and MSE of each model:
result1 = pd.DataFrame(model1.predict(X_test).flatten(), columns = ['model1'])
result2 = pd.DataFrame(model2.predict(X_test).flatten(), columns = ['model2'])
result3 = pd.DataFrame(model3.predict(X_test).flatten(), columns = ['model3'])
result4 = pd.DataFrame(model4.predict(X_test).flatten(), columns = ['model4'])

# объединяем результаты моделей и добавляем цель
Summary = pd.concat([result1, result2, result3, result4, y_test], axis=1, join='inner')

# вычисляем СКО для каждой модели и каждой цели
Summary['MSE1'] = (Summary['model1'] - Summary['target'])**2
Summary['MSE2'] = (Summary['model2'] - Summary['target'])**2
Summary['MSE3'] = (Summary['model3'] - Summary['target'])**2
Summary['MSE4'] = (Summary['model4'] - Summary['target'])**2

# усредняем СКО для каждой модели
MSE_model1 = Summary['MSE1'].mean()
MSE_model2 = Summary['MSE2'].mean()
MSE_model3 = Summary['MSE3'].mean()
MSE_model4 = Summary['MSE4'].mean()
print('For 100 epochs:')
print('MSE_model1 2 layers = ', MSE_model1)
print('MSE_model2 3 layers = ', MSE_model2)
print('MSE_model3 4 layers = ', MSE_model3)
print('MSE_model4 1 layers = ', MSE_model4)

For 100 epochs:
MSE_model1 2 layers =  27.80144571973068
MSE_model2 3 layers =  24.000286454284534
MSE_model3 4 layers =  24.815818562642654
MSE_model4 1 layers =  52.74714999162214


### Вывод: наилучшие результаты продемонстрировала модель 2, состоящая из:
        - 3 скрытых полносвязных слоях, 200/190/180 нейронов,
        - функцией активации 'tahn',
        - оптимизатором 'adam',
        - при функции потерь tf.keras.losses.MeanSquaredError()